In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext import data
from torchtext.data import Field
from torchtext.data import BucketIterator
from torchtext.vocab import GloVe
# from torch.utils.data import Dataset,DataLoader

import spacy
import numpy as np
import os
import random
import pandas as pd
from time import time
import math

from sklearn.model_selection import train_test_split
import json
import pathlib
import numpy as np
import gensim

import pickle
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
from torch.nn.utils import clip_grad_norm_


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
from gensim import models
model = models.KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/IRE_PROJECT/GoogleNews-vectors-negative300.bin', binary=True)

In [3]:
model = GloVe(name='6B', dim=50)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                          
100%|█████████▉| 399849/400000 [00:14<00:00, 29079.79it/s]

In [7]:
m.vectors.numpy()

array([[ 0.418   ,  0.24968 , -0.41242 , ..., -0.18411 , -0.11514 ,
        -0.78581 ],
       [ 0.013441,  0.23682 , -0.16899 , ..., -0.56657 ,  0.044691,
         0.30392 ],
       [ 0.15164 ,  0.30177 , -0.16763 , ..., -0.35652 ,  0.016413,
         0.10216 ],
       ...,
       [-0.51181 ,  0.058706,  1.0913  , ..., -0.25003 , -1.125   ,
         1.5863  ],
       [-0.75898 , -0.47426 ,  0.4737  , ...,  0.78954 , -0.014116,
         0.6448  ],
       [ 0.072617, -0.51393 ,  0.4728  , ..., -0.18907 , -0.59021 ,
         0.55559 ]], dtype=float32)

In [3]:
max_sent = -1
train_dataset={'story':[],'labels':[]}
for line in open('/content/gdrive/My Drive/IRE_PROJECT/train_data.json'):
  data=json.loads(line)
  max_sent = max(max_sent,len(data['story']))

  story = []
  for d in data['story']:
    story.append(str.lower(d))

  # print(data['story'])
  # print(data['labels'])
  train_dataset['story'].append(story)
  train_dataset['labels'].append(data['labels'])
# train=json.loads(pathlib.Path('/content/train_data.json').read_text())

In [4]:
validation_dataset={'story':[],'labels':[]}
for line in open('/content/gdrive/My Drive/IRE_PROJECT/validation_data.json'):
  data=json.loads(line)
  # print(data['story'])
  # print(data['labels'])
  story = []
  for d in data['story']:
    story.append(str.lower(d))

  max_sent = max(max_sent,len(data['story']))

  validation_dataset['story'].append(story)
  
  validation_dataset['labels'].append(data['labels'])
# train=json.loads(pathlib.Path('/content/train_data.json').read_text())

In [5]:
test_dataset={'story':[],'labels':[],'summary':[]}
max_summ = -1
for line in open('/content/gdrive/My Drive/IRE_PROJECT/test_data.json'):
  data=json.loads(line)
  # print(data)
  # print(data['story'])
  # print(data['labels'])
  story = []
  for d in data['story']:
    story.append(str.lower(d))
  summary=[]
  for d in data['summary']:
    summary.append(str.lower(d))

  max_sent = max(max_sent,len(data['story']))
  max_summ = max(max_summ,len(data['summary']))
  test_dataset['story'].append(story)
  
  test_dataset['labels'].append(data['labels'])

  test_dataset['summary'].append(summary)


In [38]:
max_summ

5

In [6]:
train_df = pd.DataFrame(train_dataset, columns=["story", "labels"])
validation_df=pd.DataFrame(validation_dataset, columns=["story", "labels"])
test_df = pd.DataFrame(test_dataset, columns=["story", "labels","summary"])

In [18]:
test_df

,story,labels,summary
0,"[washington ( cnn ) --, nearly 16 years after ...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",[the faa says boeing has failed to meet a dead...
1,"[( cnn ) --, cockiness and swagger serve him o...","[0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[northern ireland native won u.s. open by eigh...
2,"[in the world of military strategy , every con...","[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[military strategy requires examining worst-ca...
3,"[philadelphia ( cnn ) --, on a warm , late sum...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...",[the home field of the north philadelphia azte...
4,[an interactive map showing the names and addr...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[the publisher issues a statement supporting t...
...,...,...,...
395,"[( cnn ) --, a town engulfed by fighting in so...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[a decision to switch a local government headq...
396,"[san juan , puerto rico ( cnn ) --, investigat...","[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",[blaze started with massive explosion early fr...
397,"[( cnn ) --, the islamist rebels fighting to o...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",[french airstrikes hit behind islamist rebel l...
398,"[( cnn ), -- headlines and pundits once again ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",[north korea 's artillery barrage was serious ...


In [23]:
validation_df

,story,labels
0,"[( cnn ) --, sporting a black t-shirt proudly ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,[washington ( cnn ) -- how does the american p...,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, ..."
2,"[( cnn ) --, basketball star dennis rodman see...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3,"[( cnn ), -- check with your airline before yo...","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,"[riyadh , saudi arabia --, ahmad al shayea is ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
...,...,...
355,"[( cnn ) --, the world health organization rai...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
356,"[dali , guizhou , china ( cnn ) -- shi wenchan...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
357,"[cairo , egypt ( cnn ) -- one man died and ano...","[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
358,[london ( cnn ) -- the cleanup operation conti...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [19]:
# weights = model.vectors
dim = 50
weights = model.vectors.numpy()
weights = np.insert(weights, 0,np.zeros(dim),axis = 0)
weights = np.insert(weights, 0,np.zeros(dim),axis = 0)
# weights = torch.FloatTensor(weights)

In [20]:
word2id = {model.itos[i]:i+2 for i in range(len(model.itos))}
word2id['<pad>'] = 0
word2id['<unk>'] = 1

In [22]:
# np.save('/content/gdrive/My Drive/IRE_PROJECT/weights.npy',weights)

In [25]:
# with open('/content/gdrive/My Drive/IRE_PROJECT/word2id.json', 'w') as fp:
    # json.dump(word2id, fp)

In [7]:
weights = np.load('/content/gdrive/My Drive/IRE_PROJECT/weights.npy')
weights = torch.FloatTensor(weights)

In [8]:
word2id = json.load(open('/content/gdrive/My Drive/IRE_PROJECT/word2id.json'))

In [9]:
class Dataset():
    def __init__(self, data_list):
        self._data = data_list
    def __len__(self):
        return len(self._data)
        
    def __call__(self, batch_size, shuffle = True):
        max_len = len(self)
        
        indices = [i for i in range(0,max_len,batch_size)]
        
        if shuffle:
            indices = np.random.randint(0,max_len-1,math.ceil(max_len/batch_size))
            
        batchs = [self._data[index:index + batch_size] for index in indices]
        return batchs
    
    def __getitem__(self, index):
        return self._data.iloc[index,:].to_numpy()
    
class DataLoader():
    def __init__(self, dataset, batch_size = 1, shuffle = True):
        assert isinstance(dataset, Dataset)
        assert len(dataset) >= batch_size
        self.shuffle = shuffle
        self.dataset = dataset
        self.batch_size = batch_size
    def __iter__(self):
        return iter(self.dataset(self.batch_size, self.shuffle))
    
    

In [10]:
class RNN_RNN(nn.Module):
    def __init__(self, embed=None):
        super().__init__()
        

        
        V = 400002
        D = 50
        H = 50
        S = 10
        
        P_V = 250
        P_D = 10

        self.abs_pos_embed = nn.Embedding(P_V,P_D)
        self.rel_pos_embed = nn.Embedding(S,P_D)
        
        self.embed = nn.Embedding(V,D,padding_idx=0)

        if embed is not None:
            self.embed.weight.data.copy_(embed)

        self.word_RNN = nn.GRU(
                        input_size = D,
                        hidden_size = H,
                        batch_first = True,
                        bidirectional = True
                        )
        self.sent_RNN = nn.GRU(
                        input_size = 2*H,
                        hidden_size = H,
                        batch_first = True,
                        bidirectional = True
                        )
        self.fc = nn.Linear(2*H,2*H)

        # Parameters of Classification Layer
        self.content = nn.Linear(2*H,1,bias=False)
        self.salience = nn.Bilinear(2*H,2*H,1,bias=False)
        self.novelty = nn.Bilinear(2*H,2*H,1,bias=False)
        
        self.abs_pos = nn.Linear(P_D,1,bias=False)
        
        self.rel_pos = nn.Linear(P_D,1,bias=False)

        self.bias = nn.Parameter(torch.FloatTensor(1).uniform_(-0.1,0.1))

    def max_pool1d(self,x,seq_lens):

        out = []
        for index,t in enumerate(x):
            t = t[:seq_lens[index],:]
            t = torch.t(t).unsqueeze(0)
            out.append(torch.max_pool1d(t,t.size(2)))

        out = torch.cat(out).squeeze(2)
        return out
        
    def avg_pool1d(self,x,seq_lens):

        out = []
        for index,t in enumerate(x):
            t = t[:seq_lens[index],:]
            t = torch.t(t).unsqueeze(0)
            out.append(torch.avg_pool1d(t,t.size(2)))
        
        out = torch.cat(out).squeeze(2)
        return out


    def pad_doc(self,words_out,doc_lens):
        pad_dim = words_out.size(1)
        max_doc_len = max(doc_lens)
        sent_input = []
        start = 0
        for doc_len in doc_lens:
            stop = start + doc_len
            valid = words_out[start:stop]                                       # (doc_len,2*H)
            start = stop
            if doc_len == max_doc_len:
                sent_input.append(valid.unsqueeze(0))
            else:
                pad = Variable(torch.zeros(max_doc_len-doc_len,pad_dim))

                pad = pad.cuda()
                
                sent_input.append(torch.cat([valid,pad]).unsqueeze(0))          # (1,max_len,2*H)
                del pad

        sent_input = torch.cat(sent_input,dim=0)                                # (B,max_len,2*H)
        return sent_input

    def forward(self,x,doc_lens):
        sent_lens = torch.sum(torch.sign(x),dim=1).data

        x = x.cuda()
 
        x = self.embed(x)                                                      # (N,L,D)
        # word level GRU
        H = 50

        x = self.word_RNN(x)[0]                                                 # (N,2*H,L)
        
        #word_out = self.avg_pool1d(x,sent_lens)
        word_out = self.max_pool1d(x,sent_lens)

        # make sent features(pad with zeros)
        x = self.pad_doc(word_out,doc_lens)

        sent_out = self.sent_RNN(x)[0]                                           # (B,max_doc_len,2*H)
        #docs = self.avg_pool1d(sent_out,doc_lens)                               # (B,2*H)

        docs = self.max_pool1d(sent_out,doc_lens)                                # (B,2*H)
        
        probs = []
        for index,doc_len in enumerate(doc_lens):
            valid_hidden = sent_out[index,:doc_len,:]                            # (doc_len,2*H)
            doc = torch.tanh(self.fc(docs[index])).unsqueeze(0)
            s = Variable(torch.zeros(1,2*H))
            

            s = s.cuda()

            for position, h in enumerate(valid_hidden):
                h = h.view(1, -1)                                                # (1,2*H)

                abs_index = Variable(torch.LongTensor([[position]]))

                abs_index = abs_index.cuda()
                
                abs_features = self.abs_pos_embed(abs_index).squeeze(0)
                
                rel_index = int(round((position + 1) * 9.0 / doc_len))
                rel_index = Variable(torch.LongTensor([[rel_index]]))

                rel_index = rel_index.cuda()
                
                rel_features = self.rel_pos_embed(rel_index).squeeze(0)
                
                # classification layer
                content = self.content(h) 
                salience = self.salience(h,doc)
                novelty = -1 * self.novelty(h,torch.tanh(s))
                abs_p = self.abs_pos(abs_features)
                rel_p = self.rel_pos(rel_features)
                prob = torch.sigmoid(content + salience + novelty + abs_p + rel_p + self.bias)# 
                
                s = s + torch.mm(prob,h)
                probs.append(prob)
        return torch.cat(probs).squeeze()


In [11]:
import copy
class Batch:
  def __init__(self,batch,word2id):
    self.batch=copy.deepcopy(batch.to_numpy())
    self.word2id=word2id

  def preprocess(self):
    idx=0
    doc_lens = []
    features = []
    targets = []

    sent_list = []
    for doc in self.batch[:,0]:
        sent_list += doc
        doc_lens.append(len(doc))
        targets += list(self.batch[idx,1])
        idx+=1

    maxlen=0
    for sentence in sent_list:
      s=sentence.split()
      maxlen=max(maxlen,len(s))

    for sentence in sent_list:
      words = sentence.strip().split()
      sentence = [self.word2id[word] if word in self.word2id else 1 for word in words]
      sentence += [0 for _ in range(maxlen - len(sentence))]
      features.append(sentence)


    features = torch.LongTensor(features)    
    targets = torch.LongTensor(targets)
    
    return features,targets,doc_lens





In [12]:
def eval(net,data_iter,criterion):
    net.eval()
    total_loss = 0
    batch_num = 0
    for batch in data_iter:
        features,targets,doc_lens = Batch(batch,word2id).preprocess()
        features,targets = Variable(features), Variable(targets.float())
  
        targets = targets.cuda()
        
        probs = net(features,doc_lens)
        loss = criterion(probs,targets)
        
        del features
        del targets
        torch.cuda.empty_cache()
        
        total_loss += loss.cpu().data
        batch_num += 1
    loss = total_loss / batch_num
    net.train()
    return loss

In [14]:
# del net
# torch.cuda.empty_cache()
with torch.no_grad():
  net = RNN_RNN(weights)
  criterion = nn.BCELoss()  
  optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
  loss_sum = 0
  min_loss = float('Inf')

net.cuda()

RNN_RNN(
  (abs_pos_embed): Embedding(250, 10)
  (rel_pos_embed): Embedding(10, 10)
  (embed): Embedding(400002, 50, padding_idx=0)
  (word_RNN): GRU(50, 50, batch_first=True, bidirectional=True)
  (sent_RNN): GRU(100, 50, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=100, out_features=100, bias=True)
  (content): Linear(in_features=100, out_features=1, bias=False)
  (salience): Bilinear(in1_features=100, in2_features=100, out_features=1, bias=False)
  (novelty): Bilinear(in1_features=100, in2_features=100, out_features=1, bias=False)
  (abs_pos): Linear(in_features=10, out_features=1, bias=False)
  (rel_pos): Linear(in_features=10, out_features=1, bias=False)
)

In [15]:
train_loader = DataLoader(Dataset(train_df),batch_size=16)
validation_loader=DataLoader(Dataset(validation_df),shuffle=False)

 
counter=0
debug=True
net.train()
for epoch in range(5):
    t1 = time() 
    for step, docs in enumerate(train_loader):
        features,targets,doc_lens = Batch(docs,word2id).preprocess()

        features,targets = Variable(features), Variable(targets.float())
        # features = features.cuda()
        targets = targets.cuda()

        probs = net(features,doc_lens)
        loss = criterion(probs,targets)

        # del features
        del targets
        torch.cuda.empty_cache()

        optimizer.zero_grad()
        
        loss.backward()
        # print(loss.data.size())
        #print(loss.cpu().data)

        clip_grad_norm_(net.parameters(), 1.0)
        optimizer.step()
        
        
        # if debug:
        #       print('Batch ID:%d Loss:%f' %(step,loss.cpu().data))
              # continue
        if (step+1) % 75 == 0:
              cur_loss = eval(net,validation_loader,criterion)
              if cur_loss < min_loss:
                  min_loss = cur_loss
                  print('Min Loss:%f' %(min_loss))
                  torch.save(net.state_dict(), 'SummaRuNNer.pt')
    
    cur_loss = eval(net,validation_loader,criterion)
    if cur_loss < min_loss:
      min_loss = cur_loss
      print('Min Loss:%f' %(min_loss))
      torch.save(net.state_dict(), 'SummaRuNNer.pt')
    
    t2 = time()
    print("Time Taken for each epoch:- ",t2-t1) 

Min Loss:0.377916
Min Loss:0.365657
Min Loss:0.353974
Time Taken for each epoch:-  386.7997417449951
Min Loss:0.346797
Min Loss:0.340952
Min Loss:0.340456
Time Taken for each epoch:-  378.17206144332886
Min Loss:0.335843
Min Loss:0.334676
Min Loss:0.332131
Time Taken for each epoch:-  380.823548078537
Min Loss:0.330531
Time Taken for each epoch:-  379.35919308662415
Min Loss:0.329459
Min Loss:0.329010
Min Loss:0.327670
Time Taken for each epoch:-  388.20346188545227


In [17]:
# net.load_state_dict(torch.load('/content/gdrive/My Drive/IRE_PROJECT/SummaRuNNer.pt'))
net.eval()


RNN_RNN(
  (abs_pos_embed): Embedding(250, 10)
  (rel_pos_embed): Embedding(10, 10)
  (embed): Embedding(400002, 50, padding_idx=0)
  (word_RNN): GRU(50, 50, batch_first=True, bidirectional=True)
  (sent_RNN): GRU(100, 50, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=100, out_features=100, bias=True)
  (content): Linear(in_features=100, out_features=1, bias=False)
  (salience): Bilinear(in1_features=100, in2_features=100, out_features=1, bias=False)
  (novelty): Bilinear(in1_features=100, in2_features=100, out_features=1, bias=False)
  (abs_pos): Linear(in_features=10, out_features=1, bias=False)
  (rel_pos): Linear(in_features=10, out_features=1, bias=False)
)

In [51]:
test_loader=DataLoader(Dataset(test_df),shuffle=False)

actual = []
predicted = []
for batch in test_loader:
        features,targets,doc_lens = Batch(batch,word2id).preprocess()
        #features,targets,_,doc_lens = vocab.make_features(batch)
        features,targets = Variable(features), Variable(targets.float())
        # if use_gpu:
        # features = features.cuda()
        
        
        probs = net(features,doc_lens)
        # print(probs,targets)

        sentences = ""

        for indices in probs.topk(min(doc_lens[0],5))[1].cpu().data:
          sentences += (batch['story'].to_numpy()[0][indices])+'\n'
        
        predicted.append(sentences[:-1])


        actual.append('\n'.join(batch['summary'].to_numpy()[0]))
        del features
        del targets
        torch.cuda.empty_cache()
        # break

In [41]:
!pip install py-rouge

     |████████████████████████████████| 61kB 3.9MB/s 


In [43]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [52]:
import rouge


def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in ['Avg','Best']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)
    
    scores = evaluator.get_scores(predicted, actual)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
            if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
                for hypothesis_id, results_per_ref in enumerate(results):
                    nb_references = len(results_per_ref['p'])
                    for reference_id in range(nb_references):
                        print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                        print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
                print()
            else:
                print(prepare_results(results['p'], results['r'], results['f']))

Evaluation with Avg
	rouge-1:	P: 23.90	R: 49.85	F1: 31.95
	rouge-2:	P:  8.76	R: 18.73	F1: 11.79
	rouge-3:	P:  4.54	R:  9.86	F1:  6.13
	rouge-4:	P:  2.74	R:  6.01	F1:  3.71
	rouge-l:	P: 27.68	R: 51.19	F1: 35.63
	rouge-w:	P: 15.84	R: 20.28	F1: 17.50
Evaluation with Best
	rouge-1:	P: 23.90	R: 49.85	F1: 31.95
	rouge-2:	P:  8.76	R: 18.73	F1: 11.79
	rouge-3:	P:  4.54	R:  9.86	F1:  6.13
	rouge-4:	P:  2.74	R:  6.01	F1:  3.71
	rouge-l:	P: 27.68	R: 51.19	F1: 35.63
	rouge-w:	P: 15.84	R: 20.28	F1: 17.50


In [57]:
for docs in range(10):
  print("Predicted Summary:-",''.join(predicted[docs].split('.')))
  print()
  
  print("Actual Summary:-",''.join(actual[docs].split('.')))
  print('\n\n')


Predicted Summary:- nearly 16 years after a fuel tank explosion destroyed twa flight 800 , killing all 230 aboard , the federal aviation administration on friday proposed to fine boeing co $ 1357 million for failing to meet a deadline intended to prevent similar catastrophes 
the faa said boeing failed to meet a 2010 deadline to give airlines information on how to reduce fuel tank flammability , missing the deadline by 301 days for its b-747 aircraft , and by 406 days for its b-757 planes 
some 383 boeing aircraft in the united states are affected by the delays , it said 
because of the missed deadline , airlines have asked the faa for extensions to make necessary fixes , the faa said 
the july 17 , 1996 , explosion of twa 800 , a boeing 747 , was one of the deadliest accidents in aviation history , and was among the most difficult to solve 

Actual Summary:- the faa says boeing has failed to meet a deadline to prevent similar crashes as twa flight 800 
the flight crashed in july 1996 